In [4]:
# coding=utf-8
import re
import pymysql


# 数据库查询汽车之家多外观颜色图片url
def select_more_color_url_from_MySQL():
    conn = pymysql.connect(host='127.0.0.1', user='root', passwd='123456', db='MySQL', charset='utf8')
    cur = conn.cursor()
    cur.execute('USE autohome_carcolor')
    # 查多外观颜色图片url
    sql = '''
        SELECT DISTINCT id,smallpic
        FROM cars_pics_cg1_info 
    '''
    cur.execute(sql)
    result = cur.fetchall()
    cur.close()
    conn.close()
    result_list = []
    if result:
        for i in result:
            result_list.append(list(i))
    return result_list

# 正则处理图片url,提取后缀关键code
def re_get_url_code(url):
    re_result = re.compile(r'/[\w_-]+(.png|.jpg)$').search(url)
    if re_result:
        re_result = re_result.group()
        # 处理前部/
        re_result = re_result.replace('/y_', '').replace('/w_', '').replace('/t_', '').replace('/', '')
        # 处理可能出现的autohomecar__
        re_result = re_result.replace('autohomecar__', '')
        # 处理后部.jpg .png
        re_result = re_result.replace('.jpg', '').replace('.png', '')
        return re_result
    else:
        print('正则表达式无法匹配')
        return None
    
# 更新url_code至汽车之家
def autohome_update_url_code_to_MySQL(update_list):
    conn = pymysql.connect(host='127.0.0.1', user='root', passwd='123456', db='MySQL', charset='utf8')
    cur = conn.cursor()
    cur.execute('USE autohome_carcolor')
    sql = '''
        UPDATE `cars_pics_cg1_info`
        SET url_code = %s
        WHERE id = %s
    '''
    rows = cur.executemany(sql, update_list)
    conn.commit()
    cur.close()
    conn.close()
    
    
    
select_more_color_url_result = select_more_color_url_from_MySQL()
print(len(select_more_color_url_result))

update_list1 = []
for j in select_more_color_url_result:
    n_id = j[0]
    smallpic = j[1]
    # 正则处理
    re_result1 = re_get_url_code(smallpic)
    update_list1.append((re_result1,n_id))

# 更新
print('开始更新')
autohome_update_url_code_to_MySQL(update_list1)




111793
开始更新
